# Group Chat

As your application grows bigger, you may want to create more than one agent, each with a different set of tools and purpose to handle different part of the problem. One common pattern of utilizing multiple agents is to create a group chat between a producer and reviewer, where the two iteratively improve the generated answer.

## Group Chat to write test problems and solutions according to Bloom’s Taxonomy of Educational Objectives
We will use a group chat to orchestrate a conversation between a two agents:
- test_writer: writes a set of questions and marking scheme about a subject
- reviewer: apply [Bloom’s Taxonomy of Educational Objectives](https://en.wikipedia.org/wiki/Bloom%27s_taxonomy#:~:text=Bloom's%20taxonomy%20is%20a%20set,cognitive%2C%20affective%20and%20psychomotor%20domains.) to evaluate the quality of the test problems

In [1]:
from agentx.agent import Agent
from agentx.schema import GenerationConfig, Message, Content
from agentx.groupchat import astar_chat, reconstruct_path
from agentx.tool import Tool
from pydantic import BaseModel, Field
from typing import Dict, List, Literal
from functools import partial
from dotenv import load_dotenv
from rich import print as rich_print
import os

load_dotenv()

generation_config = GenerationConfig(
    api_type='azure',
    api_key=os.environ.get('AZURE_OPENAI_KEY'),
    base_url=os.environ.get('AZURE_OPENAI_ENDPOINT'),
    azure_deployment='gpt-35',
)

# this agent will write the question / answer pair
test_writer = Agent(
    name='test_writer',
    generation_config=generation_config,
    system_prompt='''You are an experience teacher. According to the user request and reviewer feedback, write a test question / answer pair.''',
)


In [2]:
# test test_writer agent
response = test_writer.generate_response(
    messages=[
        Message(
            role='user',
            content=Content(
                text='''The question should test student's {bloom_objective} of the topic.
Write a multiple choice question about {topic}.
Give reasoning to solve the question.
Lastly, specify the correct answer.'''.format(
                    bloom_objective='application',
                    topic = 'Conditional probability'
                ),
            ),
        ),
    ],
)

rich_print(response[-1].content.text)

Question:
A committee of 5 people is randomly selected from a group of 10 men and 5 women. What is the probability that the 
committee has at least 3 men on it?

A) 0.50
B) 0.65
C) 0.70
D) 0.80

Reasoning:
To solve this problem, we can use the concept of conditional probability. We first find the total number of ways to
choose a committee of 5 people from the group of 15 (10 men and 5 women), which is given by ${15 \choose 5}$. Then,
we find the number of ways to choose a committee with at least 3 men, which includes the following scenarios:
1. 3 men and 2 women
2. 4 men and 1 woman
3. 5 men 
We calculate the probability for each scenario and sum the probabilities to find the final answer.

Correct Answer:
C) 0.70

That's not a bad response out of the box, but can we do better? Let's try to add a reviewer to the mix.

In [3]:

# taken from wikipedia: https://en.wikipedia.org/wiki/Bloom%27s_taxonomy
bloom_objectives = {
    'knowledge': '''involves recognizing or remembering facts, terms, basic concepts, or answers without necessarily understanding what they mean. Some characteristics may include:
Knowledge of specifics—terminology, specific facts
Knowledge of ways and means of dealing with specifics—conventions, trends and sequences, classifications and categories
Knowledge of the universals and abstractions in a field—principles and generalizations, theories and structures''',
    'comprehension': 'involves demonstrating an understanding of facts and ideas by organizing, summarizing, translating, generalizing, giving descriptions, and stating the main ideas.',
    'application': 'involves using acquired knowledge to solve problems in new situations. This involves applying acquired knowledge, facts, techniques and rules. Learners should be able to use prior knowledge to solve problems, identify connections and relationships and how they apply in new situations.',
    'analysis': '''involves examining and breaking information into component parts, determining how the parts relate to one another, identifying motives or causes, making inferences, and finding evidence to support generalizations. Its characteristics include:
Analysis of elements
Analysis of relationships
Analysis of organization''',
    'synthesis': '''involves building a structure or pattern from diverse elements; it also refers to the act of putting parts together to form a whole or bringing pieces of information together to form a new meaning. Its characteristics include:
Production of a unique communication
Production of a plan, or proposed set of operations
Derivation of a set of abstract relations''',
    'evaluation': '''involves presenting and defending opinions by making judgments about information, the validity of ideas, or quality of work based on a set of criteria. Its characteristics include:
Judgments in terms of internal evidence
Judgments in terms of external criteria''',
}

bloom_reviewers = {
    objective:Agent(
        name='bloom_reviewer_specialty_{objective}'.format(objective=objective),
        generation_config=generation_config,
        system_prompt='''You specialize in reviewing {objective} questions, which {description}.
Critically access if the test question is at the right level and quality of Bloom's Taxonomy.
Then, give a score out of 10 and actionable feedback on how to improve the question.'''.format(objective=objective, description=description),
    ) for objective, description in bloom_objectives.items()
}

correctness_reviewer = Agent(
    name='correctness_reviewer',
    generation_config=generation_config,
    system_prompt='Work out the test question independently and verify the answer.',
)

In [4]:
# Test reviewer

review = bloom_reviewers['application'].generate_response(
    messages=[
        Message(
            role='user',
            content=Content(
                text=response[-1].content.text,
            ),
        )
    ]
)

In [5]:
rich_print(review[-1].content.text)

The given question is at the application level of Bloom's Taxonomy as it requires the application of the concept of
conditional probability to solve a real-world problem. The learner needs to apply their understanding of 
probability and combinations in a new context to find the probability of selecting a committee with at least 3 men 
from a given group.

I would give this question a score of 8 out of 10. 

To further improve the question, I would suggest the following:
1. Provide a clear explanation of the concept of conditional probability for the learners who might not be familiar
with it. This will help them connect the application to the underlying concept.
2. Consider providing a brief rationale for why the chosen scenarios (3 men and 2 women, 4 men and 1 woman, 5 men) 
cover all possible cases of having at least 3 men on the committee. This would help learners understand the 
comprehensiveness of the approach.

Overall, this is a well-constructed application-level question that tests the learner's ability to apply their 
understanding of probability in a new context. With the suggested improvements, it can further enhance the 
understanding and clarity for the learners.

In [6]:
# Test correctness
# From time to time the agent will write incorrect answer so it will be nice to have a checker in place.

correctness = correctness_reviewer.generate_response(
    messages=[
        Message(
            role='user',
            content=Content(
                text=response[-1].content.text,
            ),
        )
    ]
)

In [7]:
rich_print(correctness[-1].content.text)

First, let's calculate the total number of ways to choose a committee of 5 people from the group of 10 men and 5 
women:

Total ways = ${15 \choose 5}$

Now, we find the number of ways to choose a committee with at least 3 men:

1. 3 men and 2 women: ${10 \choose 3} * {5 \choose 2}$
2. 4 men and 1 woman: ${10 \choose 4} * {5 \choose 1}$
3. 5 men: ${10 \choose 5}$

The total number of ways to choose a committee with at least 3 men is the sum of the above three scenarios.

So, the probability of selecting a committee with at least 3 men is given by:
[ P = \frac{{\text{Total ways with at least 3 men}}}{{\text{Total ways}}} \]

Now, calculate the probability:
[ P = \frac{{\left( {10 \choose 3} \times {5 \choose 2} \right) + \left( {10 \choose 4} \times {5 \choose 1} 
\right) + {10 \choose 5}}}{{ {15 \choose 5}}} \]

Calculating this probability gives us P = 0.70

So, the correct answer is C) 0.70.

Given the above reasoning and calculation, the correct answer is C) 0.70.

In [8]:
# Let's construct a group chat to chain these agents together

from agentx.groupchat import group_chat

messages, _ = await group_chat(
    agents=[
        test_writer,
        bloom_reviewers['application'],
        correctness_reviewer,
    ],
    messages=[
        Message(
            role='user',
            content=Content(
                text='''The question should test student's {bloom_objective} of the topic.
Write a multiple choice question about {topic}.
Give reasoning to solve the question.
Lastly, specify the correct answer.'''.format(
                    bloom_objective='application',
                    topic = 'Conditional probability'
                ),
            ),
        ),
    ],
    max_iteration=10,
)


In [18]:
# print the last response which the test_writer agent gave
rich_print([message for message in messages if message.name == 'test_writer'][-1].content.text)

The question has been reviewed and is ready for use:
Question:
In a survey of 100 people, it was found that 45 people own a car and 30 people own a motorcycle. Out of those who 
own a car, 25 also own a motorcycle. What is the probability that a randomly selected person owns both a car and a 
motorcycle?

A) 0.15
B) 0.25
C) 0.30
D) 0.50

Reasoning to solve the question:
To find the probability that a randomly selected person owns both a car and a motorcycle, we need to use the 
formula for conditional probability. Conditional probability is the probability of an event occurring given that 
another event has already occurred. In this case, the probability of owning both a car and a motorcycle is given by
the number of people who own both divided by the total number of people surveyed. So, we can calculate the 
conditional probability using the formula:
P(A | B) = P(A and B) / P(B)
Where P(A | B) is the probability of owning both a car and a motorcycle, P(A and B) is the number of people who own
both, and P(B) is the total number of people surveyed who own a car.

Correct answer:
B) 0.25

In [21]:
# Convert the response to a JSON format for downstream processing
# e.g. render into a pdf, save to a database, and so on.

class TestQuestion(BaseModel):
    question:str
    choices:List[str]
    answer:str
    explanation:str
    bloom_objective:List[Literal['knowledge', 'comprehension', 'application', 'analysis', 'synthesis', 'evaluation']]

test_question_extractor = Agent(
    name='test_question_extractor',
    generation_config=generation_config,
    system_prompt='''Extract the latest test question. You must reply an JSON object.''',
)

test_question = test_question_extractor.generate_response(
    messages=[[message for message in messages if message.name == 'test_writer'][-1]],
    output_model=TestQuestion,
)[-1].content.text

test_question = TestQuestion.model_validate_json(test_question)

In [22]:
rich_print(test_question)

TestQuestion(
    question='In a survey of 100 people, it was found that 45 people own a car and 30 people own a motorcycle. Out 
of those who own a car, 25 also own a motorcycle. What is the probability that a randomly selected person owns both
a car and a motorcycle?',
    choices=['A) 0.15', 'B) 0.25', 'C) 0.30', 'D) 0.50'],
    answer='B) 0.25',
    explanation='To find the probability that a randomly selected person owns both a car and a motorcycle, we need 
to use the formula for conditional probability. Conditional probability is the probability of an event occurring 
given that another event has already occurred. In this case, the probability of owning both a car and a motorcycle 
is given by the number of people who own both divided by the total number of people surveyed. So, we can calculate 
the conditional probability using the formula: P(A | B) = P(A and B) / P(B) Where P(A | B) is the probability of 
owning both a car and a motorcycle, P(A and B) is the number of people who own both, and P(B) is the total number 
of people surveyed who own a car.',
    bloom_objective=['application']
)

# A* Chat
Orchestrating a conversation between a large number of agents can be challenging, especially when it is difficult to determind which agent should be called at each timestep to achieve the goal.

Seeing multi-agent chat as a path finding problem, where each message in a message history is analogous to a node, A* algorithm can be used to find the optimal path to the end of the conversation.

A* algorithm is a path finding algorithm that is widely used. It is a variant of Dijkstra's algorithm, which is used to find the shortest path between two nodes in a graph. A* algorithm is an extension of Dijkstra's algorithm, which adds a heuristic function to guide the search towards the goal. The heuristic function is an estimation of the distance between the current node and the goal. The algorithm will always choose the node with the lowest cost, which is the sum of the distance from the start node to the current node and the heuristic function.

Using A* Chat, instead of having to manually program agent behaviours, you can simply define the heuristic function that estimates how *close* the message history to the goal, and the algorithm will automatically orchestrate the conversation between the agents to reach the goal.